Clean and Analyze CDC Data 

In [67]:
import pandas as pd

# Load Data
cig_data = pd.read_csv("/Users/kathrynmawhinney/Documents/GitHub/Homework3/data/input/CDC_1970-2018.csv")
cpi_data = pd.read_excel("/Users/kathrynmawhinney/Documents/GitHub/Homework3/data/input/CPI_1913_2019.xlsx", skiprows=11)

# Clean Tobacco Data
measure_mapping = {
    "Average Cost per pack": "cost_per_pack",
    "Cigarette Consumption (Pack Sales Per Capita)": "sales_per_capita",
    "Federal and State tax as a Percentage of Retail Price": "tax_percent",
    "Federal and State Tax per pack": "tax_dollar",
    "Gross Cigarette Tax Revenue": "tax_revenue",
    "State Tax per pack": "tax_state"
}

cig_data["measure"] = cig_data["SubMeasureDesc"].map(measure_mapping)

cig_data = cig_data.rename(columns={
    "LocationAbbr": "state_abb",
    "LocationDesc": "state",
    "Data_Value": "value"
}).filter(["state_abb", "state", "Year", "measure", "value"])

# Pivot Tobacco Data
final_data = cig_data.pivot(index=["state", "Year"], columns="measure", values="value").reset_index()

# Clean CPI Data
cpi_data = cpi_data.melt(id_vars=["Year"], 
                          value_vars=["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"], 
                          var_name="month", 
                          value_name="index")

cpi_data = cpi_data.groupby("Year", as_index=False).agg({"index": "mean"})

# Merge CPI Data and Adjust Prices to 2010 Dollars
final_data = final_data.merge(cpi_data, on="Year", how="left")
final_data["price_cpi"] = final_data["cost_per_pack"] * (218 / final_data["index"])

# Save Processed Data
final_data.to_csv("/Users/kathrynmawhinney/Documents/GitHub/Homework3/data/output/TaxBurden_Data.txt", sep="\t", index=False)
final_data.to_pickle("/Users/kathrynmawhinney/Documents/GitHub/Homework3/data/output/TaxBurden_Data.pkl")


/opt/anaconda3/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
